In [21]:
import pandas as pd
import requests
import json

df = pd.read_csv('Data.csv')

rates = []

json_data = {}


for _, row in df.iterrows():
    
    hotels_id = row['hotels_id']
    check_in = row['check-in']
    check_out = row['check-out']

    
    url = f"https://www.qantas.com/hotels/api/ui/properties/{hotels_id}/availability?checkIn={check_in}&checkOut={check_out}&adults=2&children=0&infants=0&payWith=cash"
    print(f"Constructed URL: {url}")

    
    headers = {
        'Referer': url  
    }

    
    response = requests.get(url, headers=headers)


    if response.status_code == 200:

        
        json_data = response.json()

        
        for room_type in json_data['roomTypes']:
            
            for offer in room_type.get('offers', []):
                
                rate = {
                    "Room_name": room_type.get("name"),
                    "Rate_name": offer.get("name"),
                    "Number_of_Guests": room_type.get("maxOccupantCount"),
                    "Cancellation_Policy": offer.get("cancellationPolicy", {}).get("description"),
                    "Price": offer.get("charges", {}).get("total", {}).get("amount"),
                    "Top_Deal": offer.get("promotion") is not None and offer.get("promotion").get("name", "") == "Top Deal",
                    "Currency": offer.get("charges", {}).get("total", {}).get("currency")
                }
                
                rates.append(json.dumps(rate))
                print(json.dumps(rate, indent=4, sort_keys=True))

    else:
        print(f"Error: Unable to fetch data. Status code {response.status_code}")

Constructed URL: https://www.qantas.com/hotels/api/ui/properties/18482/availability?checkIn=2023-12-01&checkOut=2023-12-02&adults=2&children=0&infants=0&payWith=cash
{
    "Cancellation_Policy": "This property imposes the following penalties to its customers that Qantas Hotels is required to pass on: Cancellations or changes made after 14:00 (GMT+00:00) on Nov 30, 2023 are subject to a 1 night room & tax penalty. If you fail to check-in for this reservation, or if you cancel or change this reservation after check-in, you may incur penalty charges at the discretion of the property of up to 100% of the booking value. If you redeem Qantas Points or a Qantas Voucher please refer to the relevant Important Information and Terms & Conditions on the checkout page.",
    "Currency": "AUD",
    "Number_of_Guests": 2,
    "Price": "483.43",
    "Rate_name": "Internet Included - Non-refundable",
    "Room_name": "Double Room (Compact)",
    "Top_Deal": false
}
{
    "Cancellation_Policy": "This pr